<a href="https://colab.research.google.com/github/rohbot25/Erohan_Repository/blob/main/SoilSim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install taichi==1.1.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.5 MB/s eta 0:00:00


In [ ]:
import taichi as ti
import numpy as np
import os
import math

mean = 0.5
std = 0.25

max_steps = 200
dt = 0.001
learning_rate = 0.1
gravity = -9.8
damping = 0.99
stiffness = 1000.0
ground_height = 0.1
sim_size = 512
epsilon = 0.00001

soil_particles = 100

real = ti.f32

soil = []
mass = []
hill = np.random.normal(mean,std,soil_particles)
print(hill)
for i in range(soil_particles):
  x = np.random.uniform(0.5,1)
  y = np.random.uniform(0.1,0.2)
  soil.append([x,y])
  #one hill
  #soil.append([hill[i],np.random.uniform()* 0.3 + 0.1])
  mass.append(int(np.random.random() * 5) + 5)

#soil.append([0.5,0.5])
#soil.append([0.45,0.5])
#mass.append(5)
#mass.append(5)
ti.init( default_fp = real )

vec = lambda: ti.Vector.field(2, dtype = real)

soilPositions = vec()
soilVelocities = vec()
masses = ti.field(ti.i32)

# Declare fields for variables to debug
normals = vec()
tangents = vec()
v1ns = ti.field(dtype=real)
v1ts = ti.field(dtype=real)
v2ns = ti.field(dtype=real)
v2ts = ti.field(dtype=real)
v1n2s = ti.field(dtype=real)
v2n2s = ti.field(dtype=real)
vec1ns = vec()
vec1ts = vec()
vec2ns = vec()
vec2ts = vec()
overlaps = ti.field(dtype=real)

ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(normals)
ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(tangents)
ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(v1ns)
ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(v1ts)
ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(v2ns)
ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(v2ts)
ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(v1n2s)
ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(v2n2s)
ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(vec1ns)
ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(vec1ts)
ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(vec2ns)
ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(vec2ts)
ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(overlaps)


ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(soilPositions)
ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(soilVelocities)
ti.root.dense(ti.i, soil_particles).place(masses)


# ------------------------------------------

def Draw(frameOffset):
  for timeStep in range(0,max_steps):

    gui = ti.GUI("Robot" , (512,512), background_color = 0xFFFFFF, show_gui=False)

    #draw the floor
    gui.line( begin = (0,0.2) ,
              end = (0.5,0.2) ,
              color = 0x0,
              radius = 1)
    gui.line( begin = (0.5,0.2) ,
              end = (0.5,0.1) ,
              color = 0x0,
              radius = 1)
    gui.line( begin = ( 0.5, ground_height) ,
              end = (1,ground_height) ,
              color = 0x0,
              radius = 1)
    gui.line( begin = (0,0) ,
              end = (0,1) ,
              color = 0x0,
              radius = 1)
    gui.line( begin = (1,0) ,
              end = (1,1) ,
              color = 0x0,
              radius = 1)
    #draw the soil
    for particle in range(soil_particles):
      x = soilPositions[timeStep,particle][0]
      y = soilPositions[timeStep,particle][1]
      gui.circle ( (x,y), color = 0x0 , radius = masses[particle] )

    gui.show( 'test' + str(frameOffset + timeStep) + '.png')

# ----------------------------------------
def Initialize():

  for objectIndex in range(soil_particles):

    soilPositions[0,objectIndex] = soil[objectIndex]

    soilVelocities[0,objectIndex] = [0.0,0.0]

    masses[objectIndex] = mass[objectIndex]

# ----------------------------------------
def Simulate():

  for timeStep in range(1,max_steps):

    Step_One(timeStep)
#----------------------------------------

@ti.kernel
def Simulate_Objects(timeStep: ti.i32):

  for objA in range(soil_particles):

    oldPosition = soilPositions[timeStep-1, objA]

    newVelocity = soilVelocities[timeStep-1, objA] + \
    dt * gravity * ti.Vector([0,1])

    newPosition = oldPosition + dt * newVelocity
    collision = False
    for objB in range(soil_particles):
      if (objB != objA):
        posA = soilPositions[timeStep-1,objA]
        posB = soilPositions[timeStep-1,objB]
        dist = posA - posB
        length = dist.norm()

        radiusA = masses[objA] / sim_size
        radiusB = masses[objB] / sim_size
        radiiSum = radiusA + radiusB
        if (length < radiiSum):
          collision = True
          if length < epsilon:
            length = epsilon

          normal = dist / length
          tangent = ti.Vector([-normal[1], normal[0]])

          v1n = ti.Vector.dot(newVelocity, normal)
          v1t = ti.Vector.dot(newVelocity, tangent)
          v2n = ti.Vector.dot(soilVelocities[timeStep-1,objB], normal)
          v2t = ti.Vector.dot(soilVelocities[timeStep-1,objB], tangent)

          v1n2 = (v1n * (masses[objA] - masses[objB]) + 2 * masses[objB] * v2n) / (masses[objA] + masses[objB])
          v2n2 = (v2n * (masses[objB] - masses[objA]) + 2 * masses[objA] * v1n) / (masses[objA] + masses[objB])

          vec1n = v1n2 * normal
          vec1t = v1t * tangent
          vec2n = v2n2 * normal
          vec2t = v2t * tangent

          soilVelocities[timeStep,objA] = (vec1n + vec1t) * (1-damping)
          soilVelocities[timeStep,objB] = (vec2n + vec2t) * (1-damping)
          overlap = (radiiSum - length) / 2
          newPosition = soilPositions[timeStep-1,objA] + overlap * normal
          soilPositions[timeStep, objB] = soilPositions[timeStep-1,objB] - overlap * normal

          while length < radiiSum:
            newPosition += overlap * normal
            soilPositions[timeStep, objB] -= overlap * normal
            length += overlap * 2

                    # Store variables for debugging
          normals[timeStep, objA][0] = normal[0]
          normals[timeStep, objA][1] = normal[1]
          tangents[timeStep, objA][0] = tangent[0]
          tangents[timeStep, objA][1] = tangent[1]
          v1ns[timeStep, objA] = v1n
          v1ts[timeStep, objA] = v1t
          v2ns[timeStep, objA] = v2n
          v2ts[timeStep, objA] = v2t
          v1n2s[timeStep, objA] = v1n2
          v2n2s[timeStep, objA] = v2n2
          vec1ns[timeStep, objA][0] = vec1n[0]
          vec1ns[timeStep, objA][1] = vec1n[1]
          vec1ts[timeStep, objA][0] = vec1t[0]
          vec1ts[timeStep, objA][1] = vec1t[1]
          vec2ns[timeStep, objA][0] = vec2n[0]
          vec2ns[timeStep, objA][1] = vec2n[1]
          vec2ts[timeStep, objA][0] = vec2t[0]
          vec2ts[timeStep, objA][1] = vec2t[1]
          overlaps[timeStep, objA] = overlap

    particle_radius = masses[objA] / sim_size
    if newPosition[0] > 1 - particle_radius:
          newPosition[0] = 1 - particle_radius
          newVelocity[0] *= -1 * (1-damping)

    if newPosition[0] < particle_radius:
          newPosition[0] = particle_radius
          newVelocity[0] *= -1 * (1-damping)

    if newPosition[1] < ground_height + particle_radius:
          newPosition[1] = ground_height + particle_radius
          newVelocity[1] *= -1 * (1-damping)

    if not collision:
      soilVelocities[timeStep,objA] = newVelocity
      soilPositions[timeStep,objA] = newPosition


def Step_One(timeStep: ti.i32):

  Simulate_Objects(timeStep)


# ----------------------------------------

def Make_Movie():

  os.system("rm movie.mp4")
  os.system(" ffmpeg -i test%d.png movie.mp4")

# ----------------------------------------


# ---------- Main body of code

Initialize()

Simulate()

os.system("rm *.png")

Draw(0)


Make_Movie()

# Debug printing function
def print_debug_info():
    for timeStep in range(1, max_steps):
        for objA in range(soil_particles):
            print(f"Time Step: {timeStep}, Object: {objA}")
            print(f"Normal: {normals[timeStep, objA]}")
            print(f"Tangent: {tangents[timeStep, objA]}")
            print(f"v1n: {v1ns[timeStep, objA]}")
            print(f"v1t: {v1ts[timeStep, objA]}")
            print(f"v2n: {v2ns[timeStep, objA]}")
            print(f"v2t: {v2ts[timeStep, objA]}")
            print(f"v1n2: {v1n2s[timeStep, objA]}")
            print(f"v2n2: {v2n2s[timeStep, objA]}")
            print(f"vec1n: {vec1ns[timeStep, objA]}")
            print(f"vec1t: {vec1ts[timeStep, objA]}")
            print(f"vec2n: {vec2ns[timeStep, objA]}")
            print(f"vec2t: {vec2ts[timeStep, objA]}")
            print(f"Overlap: {overlaps[timeStep, objA]}")
            print(f"Updated Position: {soilPositions[timeStep, objA]}")
            print(f"Updated Velocity: {soilVelocities[timeStep, objA]}")
            print("------------------")

# Call this function after Make_Movie() in your main code
#print_debug_info()

#watch movie
from IPython.display import HTML
from base64 import b64encode
mp4 = open('movie.mp4', 'rb').read()
data_url = "data:video/mp4;base64,"+ b64encode(mp4).decode()

HTML('<video width=sim_size controls> <source src="%s" type="video/mp4"></video>' % data_url)

[Taichi] version 1.1.2, llvm 10.0.0, commit f25cf4a2, linux, python 3.10.12
[ 0.71479064  0.13750459  0.13367764  0.47174653  0.6635006   0.69673194
  0.67098444  0.2558558   0.56935863  0.40390504  0.38591849  0.34611814
  0.59088387  0.20615417  0.608592    0.60762578  0.61134248  0.27281197
  0.13401696  0.67173472  0.21962773  0.56969801  0.39394823  0.60396576
  0.41144444  0.59578288  0.50930978  0.91261589 -0.05561517  0.23453298
  0.08672233 -0.06519517  0.5359227   0.26068628  0.62975607  0.33579856
  0.86691719  0.77931199  0.33219252  0.76797954  0.33575224  0.70283751
  0.5048827   0.12081979  0.35354599  0.2231233   0.80800749  0.33950011
  0.33547062  0.47864878  0.79835532  0.56844901  0.36817094  0.38384457
  0.0810447   0.56253696  0.6680778   0.5414016   0.32097989  0.8825071
  0.17260561  0.58176261  0.65436896  0.43658368  0.53465575  0.38250069
  0.47429598  0.23245279  0.17194677  0.24339369  0.09615447  0.56016935
  0.42501831  0.81769853  0.58738309  0.2755543  